In [1]:
import pandas as pd
import os
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
rna_folder = '../Data/RNA'
dna_folder = '../Data/DNA'

rna_files = os.listdir(rna_folder)
dna_files = os.listdir(dna_folder)

#### Map DNA to RNA file

In [3]:
terra_files_list = []
terra_folder = '../Data/Terra_download_commands/'
for file in os.listdir(terra_folder):
    if file.endswith('.tsv'):
        terra_df = pd.read_csv(os.path.join(terra_folder, file), sep = '\t')
        if file.startswith('LUAD'):
            terra_df['DNA_file'] = terra_df['maf_file_capture_oxoG_filtered'].str.split('/').str[-1]
        else:
            terra_df['DNA_file'] = terra_df['maf_file_capture_novo_realign_filtered'].str.split('/').str[-1]
        terra_df['RNA_file'] = terra_df['maf_file_rna_final_paper_v1_3'].str.split('/').str[-1]
        terra_files_list.append(terra_df[['DNA_file', 'RNA_file']])

terra_df = pd.concat(terra_files_list)
terra_df = terra_df[((~terra_df['DNA_file'].isna()) & (~terra_df['RNA_file'].isna()))]

# Initialize a dictionary where each value is a list
dna_to_rna = defaultdict(list)

# Populate the dictionary
for dna, rna in zip(terra_df['DNA_file'], terra_df['RNA_file']):
    dna_to_rna[dna].append(rna)

# Convert defaultdict back to a regular dict if needed
dna_to_rna = dict(dna_to_rna)

#### Create a merged dataframe for DNA and RNA data

In [4]:
merged_df_list = []
chosen_columns = ['Hugo_Symbol', 'Entrez_Gene_Id', 'Tumor_Sample_Barcode', 'Chromosome', 'Start_position', 'Variant_Classification', 'Reference_Allele', 'Tumor_Seq_Allele2', 'Transcript_Exon', 'Transcript_Position', 'cDNA_Change', 'Codon_Change', 'Protein_Change', 'COSMIC_tissue_types_affected', 'COSMIC_total_alterations_in_gene', 'ref_context', 'gc_content', 'i_COSMIC_n_overlapping_mutations', 'i_init_t_lod', 'i_t_lod_fstar','t_alt_count','t_ref_count','i_tumor_f']

for dna_file in dna_to_rna:
    for rna_file in dna_to_rna[dna_file]:
        # if str(rna_file) == 'nan':
        #     print(dna_file, "doesn't exist in RNA data")
        #     continue
        if rna_file.endswith('.txt'):
            rna_sample_df = pd.read_csv(os.path.join(rna_folder, rna_file), sep = '\t', encoding = 'latin1', usecols = chosen_columns)
        if dna_file.endswith('.maf.annotated'):
            dna_sample_df = pd.read_csv(os.path.join(dna_folder, dna_file), sep = '\t', encoding = 'latin1',  skiprows=3, usecols = chosen_columns)
            dna_sample_df['i_t_lod_fstar'].astype(float)
        elif dna_file.endswith('.maf'):
            dna_sample_df = pd.read_csv(os.path.join(dna_folder, dna_file), sep = '\t', encoding = 'latin1', skiprows=1, usecols = chosen_columns)
        elif dna_file.endswith('.txt'):
            dna_sample_df = pd.read_csv(os.path.join(dna_folder, dna_file), sep = '\t', encoding = 'latin1', skiprows=1, usecols = chosen_columns)
        # elif file.endswith('.txt'):
        #     dna_sample_df = pd.read_csv(os.path.join(dna_folder, dna_file), sep = '\t', encoding = 'latin1', skiprows=1, usecols = chosen_columns)

        # dna_sample_df = dna_sample_df[chosen_columns]
        # rna_sample_df = rna_sample_df[chosen_columns]
        dna_sample_df.index = dna_sample_df['Chromosome'].astype(str) + '_' + dna_sample_df['Start_position'].astype(str) + '_' + dna_sample_df['Tumor_Seq_Allele2'].astype(str) 
        rna_sample_df.index = rna_sample_df['Chromosome'].astype(str) + '_' + rna_sample_df['Start_position'].astype(str) + '_' + rna_sample_df['Tumor_Seq_Allele2'].astype(str) 
        merged_sample_df = dna_sample_df.merge(rna_sample_df, left_index=True, right_index=True, how='outer', suffixes=['_DNA', '_RNA'])
        merged_df_list.append(merged_sample_df)

merged_df = pd.concat(merged_df_list)
for col in ['i_init_t_lod_DNA','i_t_lod_fstar_DNA','t_alt_count_DNA', 't_ref_count_DNA', 'i_tumor_f_DNA']:
    merged_df[col] = (merged_df[col].astype(str) + '|').str.split('|').str[0].astype(float)
    merged_df = merged_df[merged_df['Tumor_Seq_Allele2_DNA'].str.len() == 1]
merged_df.to_csv('../Output/DNA_RNA_merged.csv')

C:\Users\KerenYlab.MEDICINE\AppData\Local\Temp\ipykernel_4120\1814105258.py:12: DtypeWarning: Columns (39,40,41,58) have mixed types. Specify dtype option on import or set low_memory=False.
  dna_sample_df = pd.read_csv(os.path.join(dna_folder, dna_file), sep = '\t', encoding = 'latin1',  skiprows=3, usecols = chosen_columns)
C:\Users\KerenYlab.MEDICINE\AppData\Local\Temp\ipykernel_4120\1814105258.py:12: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  dna_sample_df = pd.read_csv(os.path.join(dna_folder, dna_file), sep = '\t', encoding = 'latin1',  skiprows=3, usecols = chosen_columns)
C:\Users\KerenYlab.MEDICINE\AppData\Local\Temp\ipykernel_4120\1814105258.py:12: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  dna_sample_df = pd.read_csv(os.path.join(dna_folder, dna_file), sep = '\t', encoding = 'latin1',  skiprows=3, usecols = chosen_columns)
C:\Users\KerenYlab.MEDICINE\AppData\

### Format the data to create a simple predicting model for appearance of mutations in the RNA
---
General cancer expression data is calculated in Analyze_Expression_Data.py

In [42]:
merged_df = pd.read_csv('../Output/DNA_RNA_merged.csv', index_col = 0)

C:\Users\KerenYlab.MEDICINE\AppData\Local\Temp\ipykernel_23168\4275542866.py:1: DtypeWarning: Columns (3,26) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv('../Output/DNA_RNA_merged.csv', index_col = 0)


In [5]:
merged_df['Entrez_Gene_Id_DNA'] = merged_df['Entrez_Gene_Id_DNA'].fillna(-1).astype(int).astype(str)
merged_df = merged_df[~merged_df['Start_position_DNA'].isna()]
merged_df['Appears_in_rna'] = (~merged_df['Start_position_RNA'].isna()).astype(int)
merged_df = merged_df.iloc[:,~merged_df.columns.str.endswith('RNA')]
merged_df['Cancer_type'] = merged_df['Tumor_Sample_Barcode_DNA'].str.split('-').str[0]

In [37]:
df = pd.read_csv('../Output/model_input.csv', index_col=0)
expression_df = pd.read_csv('../Output/Expression.csv', index_col=0)

In [33]:
mean_expression_by_cancer = expression_df.drop('Tumor_Sample_Barcode_DNA', axis = 1).groupby('Cancer_type').mean()

# Step 2: Map mean expression values to the df dataframe
# Assuming 'cancer_type' is a column in df as well
# df['Mean_Expression'] = df['cancer_type'].map(mean_expression_by_cancer)


In [18]:
expression_df = pd.read_csv('../Output/Expression.csv', index_col=0)

In [19]:
cancer_type = expression_df['Cancer_type']
sample = expression_df['Tumor_Sample_Barcode_DNA']
expression_df = expression_df.drop(['Cancer_type', 'Tumor_Sample_Barcode_DNA'], axis = 1)
expression_df.loc['Mean_Expression'] = expression_df.mean()

In [16]:
expression_df.mean(axis = 1)

TCGA-2F-A9KO-01A-11R-A38B-07    954.729786
TCGA-2F-A9KP-01A-11R-A38B-07    910.201528
TCGA-2F-A9KQ-01A-11R-A38B-07    914.055868
TCGA-2F-A9KR-01A-11R-A38B-07    927.736279
TCGA-2F-A9KT-01A-11R-A38B-07    945.576995
                                   ...    
TCGA-VQ-AA6I-01A-11R-A414-31    930.320116
TCGA-VQ-AA6J-01A-11R-A414-31    888.309199
TCGA-VQ-AA6K-01A-11R-A414-31    945.986212
TCGA-ZA-A8F6-01A-23R-A36D-31    904.497547
TCGA-ZQ-A9CR-01A-11R-A39E-31    951.395712
Length: 2866, dtype: float64

In [264]:
merged_df['Cancer_type'].value_counts()

Cancer_type
SKCM    402436
LUAD    186978
BLCA    136561
STAD    134690
COAD    119149
HNSC     66509
PAAD      7450
Name: count, dtype: int64

In [515]:
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

expression_data_folder = '../Data/Expression/'

results_dict = {}
files = os.listdir(expression_data_folder)
files = [file for file in files if not file.startswith('LUAD')]
for file in files:
    if not file.endswith('.txt'):
        continue
    cancer_type = file.split('.')[0]
    expression_df = pd.read_csv(os.path.join(expression_data_folder, file), sep = '\t', index_col = 0, header = 0, skiprows = [1])
    
    cancer_type_df = merged_df[merged_df['Cancer_type'] == cancer_type]
    expression_df.index = expression_df.index.str.split('|').str[1]
    cancer_type_df['Shorten_barcode'] = cancer_type_df['Tumor_Sample_Barcode_DNA'].str.split('-').str[1:3].map('-'.join)
    
    shorten_barcode = expression_df.columns.str.split('-').str[1:3].map('-'.join)

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    samples = np.array(cancer_type_df['Shorten_barcode'].unique())
    
    fold_results = []
    
    for train_index, test_index in kf.split(samples):
        temp_expression_df = expression_df.loc[:,~shorten_barcode.isin(samples[train_index])]
    
        # temp_expression_df['Mean_Expression'] = temp_expression_df.mean(axis = 1)
        # temp_cancer_type_df = cancer_type_df.merge(temp_expression_df[['Mean_Expression']], left_on = 'Entrez_Gene_Id_DNA', right_index=True)
        
        train_cancer_type_df = temp_cancer_type_df.loc[temp_cancer_type_df['Shorten_barcode'].isin(samples[train_index]),:]
        test_cancer_type_df = temp_cancer_type_df.loc[temp_cancer_type_df['Shorten_barcode'].isin(samples[test_index]),:]
        
        X_train, y_train = train_cancer_type_df[['Mean_Expression']].values, train_cancer_type_df['Appears_in_rna']
        X_test, y_test = test_cancer_type_df[['Mean_Expression']].values, test_cancer_type_df['Appears_in_rna']
        model = LogisticRegression()
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        y_pred_prob = model.predict_proba(X_test)[:, 1]
        
        # Evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_prob)
        
        fold_results.append({'accuracy': accuracy, 'roc_auc': roc_auc})
        
    # Compile results
    results_df = pd.DataFrame(fold_results)
    results_dict[cancer_type] = results_df

In [ ]:
# Variant types that do not exist in the RNA:  'Intron', RNA?, 'IGR', lincRNA?

In [516]:
for key,value in results_dict.items():
    print(f'Results for cancer {key}:')
    print(value)
    print('Mean accuracy: ', value['accuracy'].mean(), ', Mean roc_auc: ', value['roc_auc'].mean())
    print()

Results for cancer BLCA:
   accuracy   roc_auc
0  0.689010  0.806451
1  0.713347  0.802415
2  0.693296  0.808705
3  0.684852  0.798996
4  0.685488  0.797563
Mean accuracy:  0.6931986579422642 , Mean roc_auc:  0.8028258907794351

Results for cancer COAD:
   accuracy   roc_auc
0  0.714474  0.861973
1  0.729282  0.860628
2  0.708526  0.871761
3  0.709386  0.859770
4  0.716886  0.846626
Mean accuracy:  0.7157108081919978 , Mean roc_auc:  0.8601517115491382

Results for cancer HNSC:
   accuracy   roc_auc
0  0.685306  0.829730
1  0.712544  0.833500
2  0.704988  0.834233
3  0.700223  0.810802
4  0.980809  0.681440
Mean accuracy:  0.7567740126632092 , Mean roc_auc:  0.7979410689139673

Results for cancer PAAD:
   accuracy   roc_auc
0  0.786458  0.817521
1  0.738725  0.821986
2  0.756003  0.805413
3  0.781272  0.814206
4  0.780285  0.814211
Mean accuracy:  0.7685486066100021 , Mean roc_auc:  0.8146674373294065

Results for cancer SKCM:
   accuracy   roc_auc
0  0.799533  0.866210
1  0.787107  0.

In [104]:
expression_df = pd.read_csv('../Output/mean_expression_per_cancer.csv', index_col = 0)
expression_df.index = expression_df.index.str.split('|').str[1]
expression_df = pd.DataFrame({
    'Entrez_Gene_Id': expression_df.index.repeat(len(expression_df.columns)),
    'Cancer_type': list(expression_df.columns) * len(expression_df),
    'Expression': expression_df.values.flatten()
})


In [105]:
df = expression_df.merge(X[['Entrez_Gene_Id_DNA', 'Cancer_type', 'Appears_in_rna', 'Tumor_Sample_Barcode_DNA']], 
                    left_on = ['Entrez_Gene_Id', 'Cancer_type'],
                   right_on = ['Entrez_Gene_Id_DNA', 'Cancer_type'])

In [99]:
# Simple cross validation by mutations (and not by sample)
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

for cancer_type in df['Cancer_type'].unique():
    cancer_type_df = df[df['Cancer_type'] == cancer_type]
    # Define features (X) and target (y)
    X = cancer_type_df[['Expression']].values
    y = cancer_type_df['Appears_in_rna'].values
    
    # Standardize the feature
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Set up five-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    # Initialize the model
    model = LogisticRegression()
    
    # Cross-validation loop
    fold_results = []
    for train_index, test_index in kf.split(X):
        # Split the data into training and testing sets
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        y_pred_prob = model.predict_proba(X_test)[:, 1]
        
        # Evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_prob)
        # Generate the confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        
        fold_results.append(({'accuracy': accuracy, 'roc_auc': roc_auc}, cm))
    
    # Compile results
    results_df = pd.DataFrame(fold_results[fold[0] for fold in fold_results])
    
    print(f"Cross-validation results for {cancer_type}:")
    print(results_df)
    print("\nMean Accuracy:", results_df['accuracy'].mean())
    print("Mean ROC AUC:", results_df['roc_auc'].mean())
    print()


Cross-validation results for COAD:
   accuracy   roc_auc
0  0.717672  0.862840
1  0.717020  0.854323
2  0.717455  0.858084
3  0.714106  0.861688
4  0.723283  0.857485

Mean Accuracy: 0.7179070070897308
Mean ROC AUC: 0.8588843304093918

Cross-validation results for SKCM:
   accuracy   roc_auc
0  0.789754  0.866709
1  0.789909  0.866788
2  0.792509  0.867628
3  0.789699  0.863847
4  0.789104  0.866356

Mean Accuracy: 0.7901949124859613
Mean ROC AUC: 0.8662655544415555

Cross-validation results for STAD:
   accuracy   roc_auc
0  0.644788  0.820764
1  0.648721  0.825097
2  0.646697  0.822936
3  0.645743  0.821055
4  0.647843  0.821634

Mean Accuracy: 0.646758304696449
Mean ROC AUC: 0.8222970982316411

Cross-validation results for BLCA:
   accuracy   roc_auc
0  0.695116  0.801937
1  0.691725  0.805913
2  0.690067  0.803845
3  0.694201  0.802267
4  0.694766  0.797857

Mean Accuracy: 0.6931749865577578
Mean ROC AUC: 0.8023638046327303

Cross-validation results for HNSC:
   accuracy   roc_auc


#### Format data for full model

In [6]:
merged_df = pd.read_csv('../Output/DNA_RNA_merged.csv', index_col = 0)

C:\Users\KerenYlab.MEDICINE\AppData\Local\Temp\ipykernel_4120\4275542866.py:1: DtypeWarning: Columns (3,26) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv('../Output/DNA_RNA_merged.csv', index_col = 0)


In [7]:
merged_df['Entrez_Gene_Id_DNA'] = merged_df['Entrez_Gene_Id_DNA'].fillna(-1).astype(int).astype(str)
merged_df = merged_df[~merged_df['Start_position_DNA'].isna()]
merged_df['Appears_in_rna'] = (~merged_df['Start_position_RNA'].isna()).astype(int)
merged_df = merged_df.iloc[:,~merged_df.columns.str.endswith('RNA')]
merged_df['Cancer_type'] = merged_df['Tumor_Sample_Barcode_DNA'].str.split('-').str[0]

In [579]:
Non-features:
Tumor_Sample_Barcode_DNA

Features:
'Gene' - need to think of an embedding. Gene2Vec?
'Chromosome_DNA' - One-hot - remove some annoying mutations and make chromosomes strings
'Start_position_DNA' - Float - scaling
'Variant_Classification_DNA' - One-hot
'Reference_Allele_DNA' - One-hot - Maybe collapse by strand?
'Tumor_Seq_Allele2_DNA' - One-hot - Maybe collapse by strand? - Maybe remove mutations that are different that 'ref_context_DNA'  # merged_df[(merged_df['Reference_Allele_DNA'] != merged_df['ref_context_DNA'].str.slice(10,11).str.upper())]
'Transcript_Exon_DNA' - In logistic regression maybe turn to binning and one-hot
'Transcript_Position_DNA' - Float - scaling
'Gene Expression'	- Float

'Protein_Change_DNA' - Protein substitution, Myabe one hot, maybe reading frame instead
'COSMIC_tissue_types_affected_DNA' - One-hot, on all tissues
'COSMIC_total_alterations_in_gene_DNA' - Float

'ref_context_DNA' - One-hot of the left and right bases
'gc_content_DNA' - Float
'i_COSMIC_n_overlapping_mutations_DNA' - Float
'i_init_t_lod_DNA'- Float
'i_t_lod_fstar_DNA'- Float
't_alt_count_DNA'- Float
't_ref_count_DNA'- Float
'i_tumor_f_DNA'	- Float
'Cancer_type' - One-hot

,Hugo_Symbol_DNA,Entrez_Gene_Id_DNA,Chromosome_DNA,Start_position_DNA,Variant_Classification_DNA,Reference_Allele_DNA,Tumor_Seq_Allele2_DNA,Tumor_Sample_Barcode_DNA,Transcript_Exon_DNA,Transcript_Position_DNA,...,ref_context_DNA,gc_content_DNA,i_COSMIC_n_overlapping_mutations_DNA,i_init_t_lod_DNA,i_t_lod_fstar_DNA,t_alt_count_DNA,t_ref_count_DNA,i_tumor_f_DNA,Appears_in_rna,Cancer_type
10_101715548_T,DNMBP,23268,10,101715548.0,Silent,C,T,BLCA-2F-A9KO-TP,4.0,1774.0,...,CTGCCAAGCTCTTCTCAAACT,0.493,0.0,31.074207,40.720366,18.0,78.0,0.170213,0,BLCA
10_102738692_C,SEMA4G,57715,10,102738692.0,Missense_Mutation,G,C,BLCA-2F-A9KO-TP,7.0,1103.0,...,CTTCTTCACGGAGCGTGCCAC,0.602,0.0,-1.107030,6.309237,3.0,37.0,0.075000,1,BLCA
10_102822569_A,KAZALD1,81621,10,102822569.0,Missense_Mutation,G,A,BLCA-2F-A9KO-TP,2.0,546.0,...,CAGGGTGCGCGACGCGTGCGG,0.736,0.0,4.275782,9.656761,4.0,33.0,0.111111,0,BLCA
10_103826020_T,HPS6,79803,10,103826020.0,Silent,C,T,BLCA-2F-A9KO-TP,1.0,874.0,...,GGTTGCTGTCCCCCAGGGAGC,0.632,0.0,7.478583,16.651170,8.0,56.0,0.112903,1,BLCA
10_104160055_C,NFKB2,4791,10,104160055.0,Silent,G,C,BLCA-2F-A9KO-TP,16.0,1855.0,...,ACCTGGCGGTGATCACGGGGC,0.662,0.0,30.040048,36.305156,14.0,56.0,0.188406,1,BLCA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8_77763180_T,ZFHX4,79776,8,77763180.0,Silent,G,T,STAD-ZQ-A9CR-TP,10.0,4471.0,...,AGGCTAATGTGGAAGTAAAGA,0.403,0.0,4.208521,7.264761,3.0,21.0,0.130435,0,STAD
9_100881456_G,TRIM14,9830,9,100881456.0,Silent,C,G,STAD-ZQ-A9CR-TP,1.0,24.0,...,GGCTCCCGGTCGCCGCGCCCG,0.731,0.0,4.013856,6.362598,3.0,17.0,0.150000,1,STAD
9_32631933_A,TAF1L,138474,9,32631933.0,Silent,G,A,STAD-ZQ-A9CR-TP,1.0,3734.0,...,TAGTCCGTATGCGCACATAGG,0.433,0.0,43.645163,60.755127,29.0,137.0,0.163522,0,STAD
X_105179181_C,NRK,203447,X,105179181.0,Missense_Mutation,A,C,STAD-ZQ-A9CR-TP,21.0,3822.0,...,GATTCGTAGAAGTACCTGAGG,0.378,0.0,69.450299,74.950507,30.0,80.0,0.261682,0,STAD


#### Filter out the standard chromosomes

In [8]:
merged_df['Chromosome_DNA'] = merged_df['Chromosome_DNA'].astype(str).str.replace('.0','')
chromosomes = set([str(i) for i in range(1,23)] + ['X', 'Y'])
merged_df = merged_df[merged_df['Chromosome_DNA'].isin(chromosomes)]

#### Filter out by the variant type - need to make sure this is OK

In [10]:
variant_removal = merged_df[['Variant_Classification_DNA', 'Appears_in_rna']].groupby('Variant_Classification_DNA').mean()
non_zero_relevant = list(variant_removal[variant_removal['Appears_in_rna'] != 0].index)
merged_df = merged_df[merged_df['Variant_Classification_DNA'].isin(non_zero_relevant)] # remove RNA, and lincRNA?
merged_df['Left_flank_base'] =  merged_df['ref_context_DNA'].str.slice(9,10).str.upper()
merged_df['Right_flank_base'] =  merged_df['ref_context_DNA'].str.slice(11,12).str.upper()

In [24]:
tissues_dict = {}
for index, row in merged_df['COSMIC_tissue_types_affected_DNA'][~merged_df['COSMIC_tissue_types_affected_DNA'].isna()].str.split('|').items():
    for tissues in row:
        tissue = tissues[:tissues.index('(')]
        number = tissues[tissues.index('(')+1:tissues.index(')')]
        if tissue not in tissues_dict:
            tissues_dict[tissue] = {}
        try:
            number = int(number)
        except:
            number = 0
        tissues_dict[tissue][index] = number

In [271]:
existing_samples = merged_df['Tumor_Sample_Barcode_DNA'].str.split('-').str[1:3].map('-'.join).unique()

In [233]:
expression_df = pd.read_csv('../Output/mean_expression_per_cancer.csv', index_col = 0)
expression_df.index = expression_df.index.str.split('|').str[1]
expression_df = pd.DataFrame({
    'Entrez_Gene_Id': expression_df.index.repeat(len(expression_df.columns)),
    'Cancer_type': list(expression_df.columns) * len(expression_df),
    'Expression': expression_df.values.flatten()
})


In [236]:
merged_df.merge(expression_df,
                left_on = ['Entrez_Gene_Id_DNA', 'Cancer_type'],
                right_on = ['Entrez_Gene_Id', 'Cancer_type'], how = 'left')

,Hugo_Symbol_DNA,Entrez_Gene_Id_DNA,Chromosome_DNA,Start_position_DNA,Variant_Classification_DNA,Reference_Allele_DNA,Tumor_Seq_Allele2_DNA,Tumor_Sample_Barcode_DNA,Transcript_Exon_DNA,Transcript_Position_DNA,...,i_t_lod_fstar_DNA,t_alt_count_DNA,t_ref_count_DNA,i_tumor_f_DNA,Appears_in_rna,Cancer_type,Left_flank_base,Right_flank_base,Entrez_Gene_Id,Expression
0,DNMBP,23268,10,101715548.0,Silent,C,T,BLCA-2F-A9KO-TP,4.0,1774.0,...,40.720366,18.0,78.0,0.170213,0,BLCA,T,T,23268,893.592239
1,SEMA4G,57715,10,102738692.0,Missense_Mutation,G,C,BLCA-2F-A9KO-TP,7.0,1103.0,...,6.309237,3.0,37.0,0.075000,1,BLCA,G,A,57715,162.400803
2,KAZALD1,81621,10,102822569.0,Missense_Mutation,G,A,BLCA-2F-A9KO-TP,2.0,546.0,...,9.656761,4.0,33.0,0.111111,0,BLCA,C,A,81621,61.660977
3,HPS6,79803,10,103826020.0,Silent,C,T,BLCA-2F-A9KO-TP,1.0,874.0,...,16.651170,8.0,56.0,0.112903,1,BLCA,C,C,79803,566.412410
4,NFKB2,4791,10,104160055.0,Silent,G,C,BLCA-2F-A9KO-TP,16.0,1855.0,...,36.305156,14.0,56.0,0.188406,1,BLCA,T,A,4791,1578.509691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968891,ZFHX4,79776,8,77763180.0,Silent,G,T,STAD-ZQ-A9CR-TP,10.0,4471.0,...,7.264761,3.0,21.0,0.130435,0,STAD,T,G,79776,255.884184
968892,TRIM14,9830,9,100881456.0,Silent,C,G,STAD-ZQ-A9CR-TP,1.0,24.0,...,6.362598,3.0,17.0,0.150000,1,STAD,T,G,9830,1640.420685
968893,TAF1L,138474,9,32631933.0,Silent,G,A,STAD-ZQ-A9CR-TP,1.0,3734.0,...,60.755127,29.0,137.0,0.163522,0,STAD,T,C,138474,40.830964
968894,NRK,203447,X,105179181.0,Missense_Mutation,A,C,STAD-ZQ-A9CR-TP,21.0,3822.0,...,74.950507,30.0,80.0,0.261682,0,STAD,A,G,203447,82.048767


In [37]:
merged_df.merge(expression_df,
                left_on = ['Entrez_Gene_Id_DNA', 'Cancer_type'],
                right_on = ['Entrez_Gene_Id', 'Cancer_type'], how = 'left')

,Hugo_Symbol_DNA,Entrez_Gene_Id_DNA,Chromosome_DNA,Start_position_DNA,Variant_Classification_DNA,Reference_Allele_DNA,Tumor_Seq_Allele2_DNA,Tumor_Sample_Barcode_DNA,Transcript_Exon_DNA,Transcript_Position_DNA,...,t_alt_count_DNA,t_ref_count_DNA,i_tumor_f_DNA,Appears_in_rna,Cancer_type,Left_flank_base,Right_flank_base,Mutation,Entrez_Gene_Id,Expression
0,DNMBP,23268,10,101715548.0,Silent,C,T,BLCA-2F-A9KO-TP,4.0,1774.0,...,18.0,78.0,0.170213,0,BLCA,T,T,A[G>A]A,23268,893.592239
1,SEMA4G,57715,10,102738692.0,Missense_Mutation,G,C,BLCA-2F-A9KO-TP,7.0,1103.0,...,3.0,37.0,0.075000,1,BLCA,G,A,T[C>G]C,57715,162.400803
2,KAZALD1,81621,10,102822569.0,Missense_Mutation,G,A,BLCA-2F-A9KO-TP,2.0,546.0,...,4.0,33.0,0.111111,0,BLCA,C,A,T[C>T]G,81621,61.660977
3,HPS6,79803,10,103826020.0,Silent,C,T,BLCA-2F-A9KO-TP,1.0,874.0,...,8.0,56.0,0.112903,1,BLCA,C,C,G[G>A]G,79803,566.412410
4,NFKB2,4791,10,104160055.0,Silent,G,C,BLCA-2F-A9KO-TP,16.0,1855.0,...,14.0,56.0,0.188406,1,BLCA,T,A,T[C>G]A,4791,1578.509691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032202,ZFHX4,79776,8,77763180.0,Silent,G,T,STAD-ZQ-A9CR-TP,10.0,4471.0,...,3.0,21.0,0.130435,0,STAD,T,G,C[C>A]A,79776,255.884184
1032203,TRIM14,9830,9,100881456.0,Silent,C,G,STAD-ZQ-A9CR-TP,1.0,24.0,...,3.0,17.0,0.150000,1,STAD,T,G,C[G>C]A,9830,1640.420685
1032204,TAF1L,138474,9,32631933.0,Silent,G,A,STAD-ZQ-A9CR-TP,1.0,3734.0,...,29.0,137.0,0.163522,0,STAD,T,C,G[C>T]A,138474,40.830964
1032205,NRK,203447,X,105179181.0,Missense_Mutation,A,C,STAD-ZQ-A9CR-TP,21.0,3822.0,...,30.0,80.0,0.261682,0,STAD,A,G,C[T>G]T,203447,82.048767


In [224]:
df_index = merged_df.index
merged_df = merged_df.merge(expression_df,
                left_on = ['Entrez_Gene_Id_DNA', 'Cancer_type'],
                right_on = ['Entrez_Gene_Id', 'Cancer_type'], how = 'left')
merged_df.index = df_index

In [22]:
merged_df['COSMIC_tissue_types_affected_DNA']

10_101715548_T    central_nervous_system(1)|cervix(4)|endometriu...
10_102738692_C    breast(3)|cervix(1)|endometrium(2)|kidney(1)|l...
10_102822569_A                  endometrium(1)|ovary(1)|prostate(2)
10_103826020_T    endometrium(1)|kidney(1)|large_intestine(4)|lu...
10_104160055_C    NS(1)|endometrium(6)|haematopoietic_and_lympho...
                                        ...                        
8_77763180_T      NS(3)|breast(11)|central_nervous_system(1)|end...
9_100881456_G     central_nervous_system(1)|cervix(1)|endometriu...
9_32631933_A      breast(6)|central_nervous_system(4)|endometriu...
X_105179181_C     breast(8)|central_nervous_system(1)|endometriu...
X_135496331_A     NS(2)|breast(13)|central_nervous_system(2)|cer...
Name: COSMIC_tissue_types_affected_DNA, Length: 968896, dtype: object

In [52]:
tissues_df = pd.DataFrame(tissues_dict).fillna(0).astype(int)
# categorical_df = merged_df.merge(tissues_df, left_index=True, right_index=True, how='left')

numerical_df = pd.get_dummies(merged_df, columns=['Chromosome_DNA', 'Variant_Classification_DNA','Reference_Allele_DNA','Tumor_Seq_Allele2_DNA', 'Left_flank_base', 'Right_flank_base','Cancer_type'])
numerical_df = numerical_df.merge(tissues_df, left_index=True, right_index=True, how='left')
numerical_df['Cancer_type'] = numerical_df['Tumor_Sample_Barcode_DNA'].str.split('-').str[0]

In [53]:
drop_columns = ['Hugo_Symbol_DNA', 'ref_context_DNA', 'Transcript_Exon_DNA', 'cDNA_Change_DNA', 'Codon_Change_DNA', 'Protein_Change_DNA', 'COSMIC_tissue_types_affected_DNA']
# categorical_df = categorical_df.drop(drop_columns, axis = 1)
numerical_df = numerical_df.drop(drop_columns, axis = 1)

In [54]:
numerical_df = numerical_df.fillna(0)
# categorical_df = categorical_df.fillna(0)

In [85]:
df = pd.read_csv('../Output/Numerical_input.csv', index_col=0)


In [89]:
df[['Entrez_Gene_Id_DNA', 'Tumor_Sample_Barcode_DNA', 'Appears_in_rna', 'Cancer_type']].to_csv('../Output/Only_expression.csv')

In [73]:
df['Cancer_type'].value_counts()

Cancer_type
SKCM    369099
LUAD    171793
BLCA    126024
STAD    125095
COAD    108830
HNSC     61346
PAAD      6709
Name: count, dtype: int64

In [55]:
# categorical_df.to_csv('../Output/Categorical_input.csv')
numerical_df.to_csv('../Output/Numerical_input.csv')

In [321]:
sample = expression_df['sample']
cancer = expression_df['Cancer_type']

In [325]:
expression_df = expression_df.drop(['sample', 'Cancer_type'], axis = 1).mean()

Hybridization REF
100130426       0.084443
100133144      14.140035
100134869      14.523726
10357         152.733826
10431         962.013245
                ...     
7791         5391.243028
23140        1336.679062
26009         726.402235
387590         94.335828
389932          0.199441
Length: 20531, dtype: float64

In [292]:
# Merge expression data with the cancer type data
merged_train_df = dataframe.merge(, left_on=gene_column, right_index=True)


Hybridization REF,100130426,100133144,100134869,10357,10431,136542,155060,26823,280660,317712,...,79364,440590,79699,7791,23140,26009,387590,389932,Cancer_type,sample
TCGA-2F-A9KO-01A-11R-A38B-07,0.0000,20.4373,37.8717,123.0904,702.0408,0.0,406.9971,0.5831,0.0,0.0,...,1450.7289,5.8309,697.9592,4262.3907,1787.7551,572.5948,8.7464,0.0000,BLCA,2F-A9KO
TCGA-2F-A9KP-01A-11R-A38B-07,0.0000,16.1382,12.5759,137.8860,882.2305,0.0,182.2722,0.8323,0.0,0.0,...,1420.7241,17.0620,1526.4253,3105.2851,1467.7486,860.5909,2.9130,0.0000,BLCA,2F-A9KP
TCGA-2F-A9KQ-01A-11R-A38B-07,0.0000,13.3333,10.7742,104.6778,954.1029,0.0,224.8493,0.0000,0.0,0.0,...,873.4353,1.8544,1571.1637,3275.3825,750.5795,621.2332,3.2452,0.4636,BLCA,2F-A9KQ
TCGA-2F-A9KR-01A-11R-A38B-07,0.0000,15.3523,42.5810,146.4527,487.2101,0.0,472.9642,2.3743,0.0,0.0,...,1924.1476,1.8995,748.8600,4465.6181,910.7885,935.4815,366.1199,0.0000,BLCA,2F-A9KR
TCGA-2F-A9KT-01A-11R-A38B-07,0.0000,14.0136,17.6427,142.9621,954.7767,0.0,234.5958,0.0000,0.0,0.0,...,938.3833,6.7835,878.4624,3449.4064,850.1979,630.8649,10.1752,0.0000,BLCA,2F-A9KT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-VQ-AA6I-01A-11R-A414-31,0.0000,35.0189,30.1605,21.6463,519.9928,0.0,72.8760,0.0000,0.0,0.0,...,1032.0486,0.9621,975.7681,4006.9749,1904.8764,719.3795,37.0393,0.4810,STAD,VQ-AA6I
TCGA-VQ-AA6J-01A-11R-A414-31,0.2466,27.6770,19.2494,35.6665,709.6427,0.0,94.3317,0.4788,0.0,0.0,...,1078.5898,4.5490,663.1951,4794.6370,1623.9899,1091.2791,29.4487,0.7183,STAD,VQ-AA6J
TCGA-VQ-AA6K-01A-11R-A414-31,0.4134,28.6855,21.1378,11.5371,702.4735,0.0,221.5548,0.7067,0.0,0.0,...,1719.7880,3.1802,744.8763,4516.2544,2195.4064,546.6431,12.3675,1.0601,STAD,VQ-AA6K
TCGA-ZA-A8F6-01A-23R-A36D-31,0.0000,29.3939,15.4703,22.0386,561.9835,0.0,166.8634,0.0000,0.0,0.0,...,915.7812,2.3613,1291.2239,10290.4368,2792.2078,756.0016,138.5281,0.0000,STAD,ZA-A8F6


In [291]:
filtered_processed_df

,Entrez_Gene_Id_DNA,Start_position_DNA,Tumor_Sample_Barcode_DNA,Transcript_Position_DNA,COSMIC_total_alterations_in_gene_DNA,gc_content_DNA,i_COSMIC_n_overlapping_mutations_DNA,i_init_t_lod_DNA,i_t_lod_fstar_DNA,t_alt_count_DNA,...,fallopian_tube,gastrointestinal_tract_,genital_tract,peritoneum,placenta,vagina,vulva,parathyroid,retroperitoneum,paratesticular_tissues
10_101715548_T,23268,101715548.0,BLCA-2F-A9KO-TP,1774.0,61.0,0.493,0.0,31.074207,40.720366,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10_102738692_C,57715,102738692.0,BLCA-2F-A9KO-TP,1103.0,25.0,0.602,0.0,-1.107030,6.309237,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10_102822569_A,81621,102822569.0,BLCA-2F-A9KO-TP,546.0,4.0,0.736,0.0,4.275782,9.656761,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10_103826020_T,79803,103826020.0,BLCA-2F-A9KO-TP,874.0,11.0,0.632,0.0,7.478583,16.651170,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10_104160055_C,4791,104160055.0,BLCA-2F-A9KO-TP,1855.0,23.0,0.662,0.0,30.040048,36.305156,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8_77763180_T,79776,77763180.0,STAD-ZQ-A9CR-TP,4471.0,432.0,0.403,0.0,4.208521,7.264761,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9_100881456_G,9830,100881456.0,STAD-ZQ-A9CR-TP,24.0,9.0,0.731,0.0,4.013856,6.362598,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9_32631933_A,138474,32631933.0,STAD-ZQ-A9CR-TP,3734.0,159.0,0.433,0.0,43.645163,60.755127,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X_105179181_C,203447,105179181.0,STAD-ZQ-A9CR-TP,3822.0,76.0,0.378,0.0,69.450299,74.950507,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [288]:
filtered_processed_df.to_csv('../Output/model_input.csv')

In [162]:
filtered_processed_df.columns[filtered_processed_df.columns.str.startswith('Cancer')]

Index(['Cancer_type_BLCA', 'Cancer_type_COAD', 'Cancer_type_HNSC',
       'Cancer_type_LUAD', 'Cancer_type_PAAD', 'Cancer_type_SKCM',
       'Cancer_type_STAD'],
      dtype='object')

In [272]:
filtered_processed_df[['Cancer_type_BLCA', 'Cancer_type_COAD', 'Cancer_type_HNSC', 'Cancer_type_PAAD', 'Cancer_type_SKCM', 'Cancer_type_STAD']].sum()

Cancer_type_BLCA    128664
Cancer_type_COAD    111159
Cancer_type_HNSC    304413
Cancer_type_PAAD      6902
Cancer_type_SKCM    380742
Cancer_type_STAD    127777
dtype: int64

In [282]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

results_dict = {}
# Define features (X) and target (y)
# cancer_type_df = filtered_processed_df
# X = cancer_type_df.drop(['Appears_in_rna'], axis=1)
# X = cancer_type_df[['i_init_t_lod_DNA', 'i_t_lod_fstar_DNA', 'Cancer_type_HNSC', 'Left_flank_base_C', 'Left_flank_base_G', 'Left_flank_base_T',
#        'Right_flank_base_C', 'Right_flank_base_G', 'Right_flank_base_T', 'Expression']]
for cancer_type in ['Cancer_type_BLCA', 'Cancer_type_COAD', 'Cancer_type_HNSC', 'Cancer_type_PAAD', 'Cancer_type_SKCM', 'Cancer_type_STAD']:
    cancer_type_df = filtered_processed_df[filtered_processed_df[cancer_type]]
    X = cancer_type_df[['Expression']]
    # X = cancer_type_df.drop('Appears_in_rna', axis = 1)
    y = cancer_type_df['Appears_in_rna']
    
    # Store feature names
    feature_names = X.columns
    
    # Standardize the feature
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Set up five-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    # Initialize the model
    model = LogisticRegression()
    
    # Cross-validation loop
    fold_results = []
    top_n = 12  # Number of top coefficients to display
    
    for train_index, test_index in kf.split(X):
        # Split the data into training and testing sets
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        coefficients = model.coef_[0]  # Assuming binary logistic regression (1 class)
        
        # Get indices of the top N coefficients by absolute value
        top_indices = np.argsort(np.abs(coefficients))[-top_n:]
        
        # # Print the top coefficients and their feature names
        # print(f"Top {top_n} coefficients in this fold:")
        # for idx in reversed(top_indices):  # Reverse to show largest first
        #     print(f"Feature: {feature_names[idx]}, Coefficient = {coefficients[idx]:.4f}")
        
        # Make predictions
        y_pred = model.predict(X_test)
        y_pred_prob = model.predict_proba(X_test)[:, 1]
        
        # Evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_prob)
        fold_results.append({'accuracy': accuracy, 'roc_auc': roc_auc})
        
        # Print metrics
        # print(f"Fold ROC AUC: {roc_auc:.4f}, Accuracy: {accuracy:.4f}")
    
    # Compile results
    results_df = pd.DataFrame(fold_results)
    print(cancer_type)
    print("\nCross-validation results:")
    print(results_df)
    print("\nMean Accuracy:", results_df['accuracy'].mean())
    print("Mean ROC AUC:", results_df['roc_auc'].mean())
    results_dict[cancer_type] = results_df


Cancer_type_BLCA

Cross-validation results:
   accuracy   roc_auc
0  0.696693  0.816575
1  0.693545  0.816247
2  0.692885  0.817173
3  0.689465  0.815174
4  0.694000  0.814123

Mean Accuracy: 0.6933174833846742
Mean ROC AUC: 0.8158583266767158
Cancer_type_COAD

Cross-validation results:
   accuracy   roc_auc
0  0.729129  0.873062
1  0.720448  0.872773
2  0.729219  0.872175
3  0.728095  0.870481
4  0.730332  0.871023

Mean Accuracy: 0.7274444974048133
Mean ROC AUC: 0.8719028339889519
Cancer_type_HNSC

Cross-validation results:
   accuracy   roc_auc
0  0.936337  0.711680
1  0.936518  0.708830
2  0.935598  0.707625
3  0.936418  0.714505
4  0.936829  0.713465

Mean Accuracy: 0.936339777102338
Mean ROC AUC: 0.7112211190523832
Cancer_type_PAAD

Cross-validation results:
   accuracy   roc_auc
0  0.769732  0.827393
1  0.758870  0.842416
2  0.750000  0.809439
3  0.757971  0.820079
4  0.777536  0.809907

Mean Accuracy: 0.7628219416721762
Mean ROC AUC: 0.8218465544119408
Cancer_type_SKCM

Cross-v

In [83]:
import itertools
import json

# Define models and their relevant hyperparameters
models_hyperparameters = {
    "LogisticRegression": {},  # No hyperparameters to vary
    "RandomForest": {"n_estimators": [50, 100, 200], "max_depth": [3, 6, 9]},
    "SVM": {},  # No tunable hyperparameters provided here
    "XGBoost": {"n_estimators": [50, 100], "learning_rate": [0.01, 0.1], "max_depth": [3, 6, 9]},
    "NeuralNet_1": {"learning_rate": [0.001, 0.005, 0.01], "epochs": [10, 20, 50]},
    "NeuralNet_2": {"learning_rate": [0.001, 0.005, 0.01], "epochs": [10, 20, 50]},
    "NeuralNet_3": {"learning_rate": [0.001, 0.005, 0.01], "epochs": [10, 20, 50]}
}

# Define cancer types (including "all" for all cancers together)
cancer_types = ["all"] + list(merged_df['Cancer_type'].unique())

# Function to generate all argument combinations
def generate_arguments(models_hyperparameters, cancer_types):
    all_arguments = []

    for model, hyperparams in models_hyperparameters.items():
        # For models without specific hyperparameters, just create one argument set per cancer type
        if not hyperparams:
            for cancer_type in cancer_types:
                all_arguments.append({
                    "model": model,
                    "cancer_type": cancer_type
                })
        else:
            # Generate all combinations of hyperparameters for the current model
            keys, values = zip(*hyperparams.items())
            hyperparam_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
            for cancer_type in cancer_types:
                all_arguments.append({
                    "model": model,
                    "cancer_type": cancer_type
                })
            for hyperparam_set in hyperparam_combinations:
                for cancer_type in cancer_types:
                    argument_set = {"model": model, "cancer_type": cancer_type}
                    argument_set.update(hyperparam_set)  # Add hyperparameters to arguments
                    all_arguments.append(argument_set)

    return all_arguments

# Generate all combinations
all_arguments = generate_arguments(models_hyperparameters, cancer_types)

# Print or save the commands to run the script with these arguments
script_name = "Run_Prediction.py"  # Replace with the actual script name
commands = []
for args in all_arguments:
    command = f"{script_name}"
    for key, value in args.items():
        command += f" --{key} {value}"
    commands.append(command)

# Save commands to a text file
with open("../Script/commands.txt", "w") as f:
    f.write("\n".join(commands))

# Print a few commands for verification
print("\n".join(commands[:5]))


Run_Prediction.py --model LogisticRegression --cancer_type all
Run_Prediction.py --model LogisticRegression --cancer_type BLCA
Run_Prediction.py --model LogisticRegression --cancer_type COAD
Run_Prediction.py --model LogisticRegression --cancer_type HNSC
Run_Prediction.py --model LogisticRegression --cancer_type LUAD
